In [ ]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import function as f
from function import *
import proj_gradient_method as pcg
from proj_gradient_method import *
from numba import typeof
from numba.core import types
from numba.typed import Dict
from sklearn.model_selection import ParameterGrid
from importlib import reload
import time
from  varname.helpers import Wrapper

In [ ]:
reload(f)
reload(pcg)

In [ ]:
gamma = 1.0
kmax = 2500
tolgrad = 1e-8
tolx = 1e-8
c = 1e-4
rho = 0.7
btmax = 50
params = {'d': [3, 4, 5],
          'k': [2, 4, 6, 8, 10, 12]}

Exact derivative

In [ ]:
rng = np.random.RandomState(42)
i = 1
for d in [3, 4, 5]: 
    n = 10**d
    x = rng.randint(-5, 5, n)
    X1 = Dict.empty(
        key_type=types.unicode_type,
        value_type=types.float64[:]
    )
    X1['[0, n]'] = np.asarray([1, 5.12, 0, n], dtype='f8')
    X1 = Wrapper(X1)

    X2 = Dict.empty(
        key_type=types.unicode_type,
        value_type=types.float64[:]
    )
    X2['[0]'] = np.asarray([-5.12, 5.12, 0, 1], dtype='f8')
    X2['[n/2, n-1]'] = np.asarray([1, 5.12, 1, n], dtype='f8')
    X2 = Wrapper(X2)

    X3 = Dict.empty(
        key_type=types.unicode_type,
        value_type=types.float64[:]
    )
    X3['[0, n/2-1]'] = np.asarray([-5.12, 5.12, 0, n/2], dtype='f8')
    X3['[n/2, n-1]'] = np.asarray([1, 5.12, n/2, n], dtype='f8')
    X3 = Wrapper(X3)
        
    for X in [X1, X2, X3]:
        x_seq, f_seq, gradf_norm_seq, deltax_norm_seq, k, bt_seq = projected_gradient_bcktrck(x, X.value, gamma, kmax, 
                                                                                                tolgrad, tolx, c, 
                                                                                                rho, btmax, False, 
                                                                                                'Empty', 0)
        print(f"{i}) d = {d}, fin_diff = {False}, X = {X.name}")
        print(f"k = {k}, fk = {f_seq[-1]}, gradfk_norm = {gradf_norm_seq[-1]}, deltaxk_norm = {deltax_norm_seq[-1]}\n")
        i += 1

In [ ]:
sns.set()
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(f_seq, '-r')
ax.plot(gradf_norm_seq, '-g')
ax.plot(deltax_norm_seq, '-b')
ax.set_xscale('log', base=10)
ax.set_yscale('log', base=10)

Finite derivatives

In [ ]:
rng = np.random.RandomState(42)
i = 1
for param in ParameterGrid(params):
    d = param['d']
    n = 10**d
    x = rng.randint(-5, 5, n)

    X1 = Dict.empty(
        key_type=types.unicode_type,
        value_type=types.float64[:]
    )
    X1['[0, n]'] = np.asarray([1, 5.12, 0, n], dtype='f8')
    X1 = Wrapper(X1)

    X2 = Dict.empty(
        key_type=types.unicode_type,
        value_type=types.float64[:]
    )
    X2['[0]'] = np.asarray([-5.12, 5.12, 0, 1], dtype='f8')
    X2['[n/2, n-1]'] = np.asarray([1, 5.12, 1, n], dtype='f8')
    X2 = Wrapper(X2)

    X3 = Dict.empty(
        key_type=types.unicode_type,
        value_type=types.float64[:]
    )
    X3['[0, n/2-1]'] = np.asarray([-5.12, 5.12, 0, n/2], dtype='f8')
    X3['[n/2, n-1]'] = np.asarray([1, 5.12, n/2, n], dtype='f8')
    X3 = Wrapper(X3)
    
    k = param['k']
    h = 10**(-k) * np.linalg.norm(x)
    
    for X in [X1, X2, X3]:
        x_seq2, f_seq2, gradf_norm_seq2, deltax_norm_seq2, k2, bt_seq2 = projected_gradient_bcktrck(x, X.value, gamma, kmax, 
                                                                                              tolgrad, tolx, c, 
                                                                                              rho, btmax, True, 
                                                                                              'fw', h)
        print(f"{i}) d = {d}, fin_diff = {True}, type = fw, k = {param['k']}, X = {X.name}")
        print(f"k = {k2}, fk = {f_seq2[-1]}, gradfk_norm = {gradf_norm_seq2[-1]}, deltaxk_norm = {deltax_norm_seq2[-1]}\n")
        i += 1

In [ ]:
sns.set()
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(f_seq2, '-r')
ax.plot(gradf_norm_seq2, '-g')
ax.plot(deltax_norm_seq2, '-b')
ax.set_xscale('log', base=10)
ax.set_yscale('log', base=10)